# Turkish TTS Fine-Tuning on Google Colab

Bu notebook, Kokoro-82M modelini Türkçe için fine-tune etmek üzere hazırlanmıştır.

**Gereksinimler:**
- Runtime: GPU (T4 önerilen)
- RAM: High-RAM runtime (opsiyonel ama önerilen)
- Google Drive: Checkpoints kaydetmek için

**Tahmini Süre:**
- Debug (100 sample): ~10 dakika
- Full training (48k sample): 2-3 gün (T4 GPU ile)

## 1. GPU Kontrolü ve Runtime Ayarları

⚠️ **ÖNEMLI**: Runtime > Change runtime type > GPU (T4) seçili olmalı!

In [ ]:
# GPU kontrolü
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Google Drive Bağlantısı

Checkpoints ve TensorBoard logları Drive'a kaydedilecek (session sonlanırsa kaybolmaz)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Checkpoint dizinleri oluştur
!mkdir -p /content/drive/MyDrive/tts_checkpoints
!mkdir -p /content/drive/MyDrive/tts_runs
!mkdir -p /content/drive/MyDrive/tts_outputs

print("\n✓ Google Drive mounted successfully!")
print("  Checkpoints: /content/drive/MyDrive/tts_checkpoints")
print("  TensorBoard: /content/drive/MyDrive/tts_runs")
print("  Outputs: /content/drive/MyDrive/tts_outputs")

## 3. Projeyi GitHub'dan Klonla

**Seçenek A**: GitHub kullanıyorsanız (önerilen)

In [ ]:
# GitHub'dan klonla
import os

GITHUB_REPO = "https://github.com/samedberkanunver-kfn/tts.git"  # Buraya kendi repo URL'nizi yazın

if os.path.exists('/content/tts'):
    print("Removing old directory...")
    !rm -rf /content/tts

!git clone {GITHUB_REPO} /content/tts
%cd /content/tts
!ls -lh

**Seçenek B**: Google Drive'dan kopyala (GitHub kullanmıyorsanız)

In [ ]:
# Google Drive'dan kopyala (Seçenek B)
# Önce tts/ klasörünü Drive'a yükleyin: /content/drive/MyDrive/tts/

# !cp -r /content/drive/MyDrive/tts /content/
# %cd /content/tts
# !ls -lh

## 4. Sistem Bağımlılıklarını Kur (espeak-ng)

Türkçe phonemization için gerekli

In [ ]:
# espeak-ng kurulumu
!apt-get update -qq
!apt-get install -y espeak-ng > /dev/null 2>&1

# Kurulum kontrolü
!espeak-ng --version
print("\n✓ espeak-ng installed successfully!")

## 5. Python Bağımlılıklarını Kur

⚠️ **ÖNEMLI**: PyTorch'u upgrade ETMEYIN! Colab'daki versiyon zaten uyumlu.

In [ ]:
# PyTorch zaten Colab'da yüklü - UPGRADE YAPMAYIN!
# Sadece eksik paketleri yükle (sabit versiyonlar)

!pip install -q transformers==4.36.0
!pip install -q datasets==2.16.0
!pip install -q peft==0.7.0
!pip install -q accelerate==0.25.0
!pip install -q librosa==0.10.1
!pip install -q soundfile==0.12.1
!pip install -q scipy==1.11.4
!pip install -q phonemizer==3.2.1
!pip install -q sentencepiece==0.1.99
!pip install -q einops==0.7.0
!pip install -q matplotlib==3.8.2
!pip install -q tqdm==4.66.1
!pip install -q pyyaml==6.0.1
!pip install -q tensorboard==2.15.1
!pip install -q torchcodec  # Audio decode için gerekli

print("\n✓ All dependencies installed!")

# Kurulum kontrolü
import torch
import transformers
import datasets
import peft
import librosa
import phonemizer

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"peft: {peft.__version__}")
print(f"librosa: {librosa.__version__}")

# GPU bilgisi
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"\n🎮 GPU: {gpu_name}")
    if "V100" in gpu_name:
        print("✓ V100 detected - Using optimized settings (batch_size=16)")
    elif "T4" in gpu_name:
        print("✓ T4 detected - Using standard settings (batch_size=8)")
    elif "A100" in gpu_name:
        print("✓ A100 detected - Using high-performance settings (batch_size=32)")

## 6. Config Dosyasını Colab için Optimize Et

GPU'ya göre otomatik batch size ayarı:
- **T4**: batch_size=8 (~30-40 saat)
- **V100**: batch_size=16 (~16-20 saat) ⚡
- **A100**: batch_size=32 (~8-12 saat) 🚀

In [ ]:
import yaml
import torch

# Config'i yükle
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# GPU ayarları (MPS yerine CUDA)
config['training']['device'] = 'cuda'

# GPU'ya göre batch size optimize et
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    
    if "V100" in gpu_name:
        # V100: 2x daha güçlü
        config['training']['batch_size'] = 16
        config['training']['gradient_accumulation_steps'] = 2  # Effective: 32
        print("🎮 V100 GPU detected - Using batch_size=16")
    elif "A100" in gpu_name:
        # A100: 4x daha güçlü
        config['training']['batch_size'] = 32
        config['training']['gradient_accumulation_steps'] = 1  # Effective: 32
        print("🎮 A100 GPU detected - Using batch_size=32")
    else:
        # T4 veya diğer GPU'lar
        config['training']['batch_size'] = 8
        config['training']['gradient_accumulation_steps'] = 4  # Effective: 32
        print("🎮 T4 GPU detected - Using batch_size=8")
else:
    # CPU fallback
    config['training']['batch_size'] = 2
    config['training']['gradient_accumulation_steps'] = 16

# Mixed precision (GPU'da daha iyi çalışır)
config['training']['mixed_precision'] = True
config['training']['fp16'] = True
config['training']['bf16'] = False

# Workers (Colab için)
config['system']['num_workers'] = 2
config['system']['pin_memory'] = True  # GPU için aktif

# Checkpoint'leri Drive'a kaydet
config['training']['checkpoint_dir'] = '/content/drive/MyDrive/tts_checkpoints'
config['training']['tensorboard_dir'] = '/content/drive/MyDrive/tts_runs'

# Sık checkpoint kaydet (disconnect koruması)
config['training']['save_interval'] = 500  # Her 500 step
config['training']['keep_last_n_checkpoints'] = 5

# Inference output
config['inference']['output_dir'] = '/content/drive/MyDrive/tts_outputs'

# Kaydet
with open('config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

print("\n✓ Config updated for Colab GPU!\n")
print(f"Device: {config['training']['device']}")
print(f"Batch size: {config['training']['batch_size']}")
print(f"Gradient accumulation: {config['training']['gradient_accumulation_steps']}")
print(f"Effective batch size: {config['training']['batch_size'] * config['training']['gradient_accumulation_steps']}")
print(f"Mixed precision: {config['training']['mixed_precision']}")
print(f"Checkpoint dir: {config['training']['checkpoint_dir']}")

# Tahmini süre
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
if "V100" in gpu_name:
    print("\n⏱️ Estimated training time (100 epochs): ~16-20 hours")
elif "A100" in gpu_name:
    print("\n⏱️ Estimated training time (100 epochs): ~8-12 hours")
else:  # T4
    print("\n⏱️ Estimated training time (100 epochs): ~30-40 hours")

## 7. Dataset Test (Otomatik İndirilir)

İlk çalışmada dataset Hugging Face'ten indirilir (~1.6GB)

In [ ]:
from datasets import load_dataset

# Dataset'in bir kısmını yükle (test için)
print("Loading dataset sample...")
dataset_sample = load_dataset(
    "zeynepgulhan/mediaspeech-with-cv-tr",
    split="train[:100]"
)

print(f"\n✓ Dataset loaded: {len(dataset_sample)} samples")
print(f"\nSample item:")
sample = dataset_sample[0]
print(f"  Text: {sample['sentence']}")
print(f"  Audio: {len(sample['audio']['array'])} samples @ {sample['audio']['sampling_rate']} Hz")

# Full dataset info
full_dataset = load_dataset(
    "zeynepgulhan/mediaspeech-with-cv-tr",
    split="train"
)
print(f"\nFull training dataset: {len(full_dataset)} samples")

## 8. Phonemizer Test

In [ ]:
# Phonemizer test
!python -m src.phonemizer

print("\n✓ Phonemizer test passed!")

## 9. Quick Training Test (100 Sample - Debug Mode)

Pipeline'ın çalıştığından emin olmak için hızlı test (~10 dakika)

In [ ]:
# Debug mode: 100 sample ile test
!python -m src.train --config config.yaml --limit-samples 100

print("\n✓ Quick training test completed!")
print("If no errors, you're ready for full training!")

## 10. TensorBoard (Training İzleme)

Training sırasında loss'u canlı takip etmek için

In [ ]:
# TensorBoard başlat
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/tts_runs

# Training başladıktan sonra bu cell'i çalıştırın
# Loss grafikleri görünecek

## 11. Session Disconnect Koruması (Opsiyonel)

⚠️ Colab ücretsiz versiyonda ~12 saat sonra disconnect olur. Bu kod bunu engellemeye çalışır (garanti değil).

In [ ]:
# Disconnect koruması (dikkatli kullanın)
import time
from IPython.display import display, Javascript
import threading

def keep_alive():
    """Her 5 dakikada bir aktivite simüle et"""
    while True:
        try:
            display(Javascript('document.querySelector("colab-toolbar-button#connect").click()'))
        except:
            pass
        time.sleep(300)  # 5 dakika

# Background thread olarak çalıştır
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

print("✓ Keep-alive mechanism started")
print("⚠️ Not guaranteed to prevent all disconnects!")

## 12. FULL TRAINING (48k Samples)

⚠️ **DİKKAT**: Bu ~2-3 gün sürecek. Checkpoints Drive'a kaydedilir.

**Disconnect olursa**: Aşağıdaki "Resume Training" hücresini kullanın.

In [ ]:
# FULL TRAINING BAŞLAT
!python -m src.train --config config.yaml

print("\n" + "="*60)
print("TRAINING COMPLETED!")
print("Best model: /content/drive/MyDrive/tts_checkpoints/best_model.pt")
print("="*60)

## 13. Resume Training (Disconnect Sonrası Devam)

Session disconnect olduysa, en son checkpoint'ten devam edin:

In [ ]:
# En son checkpoint'i bul
import glob
import os

checkpoint_dir = '/content/drive/MyDrive/tts_checkpoints'
checkpoints = glob.glob(f'{checkpoint_dir}/checkpoint_step_*.pt')

if checkpoints:
    # Step numarasına göre sırala
    latest = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    print(f"Latest checkpoint found: {latest}")
    print(f"Resuming training...\n")
    
    # Training'e devam et
    !python -m src.train --config config.yaml --resume {latest}
else:
    print("No checkpoints found. Start fresh training.")

## 14. Checkpoints Listesi

In [ ]:
# Kaydedilen tüm checkpoints
!ls -lh /content/drive/MyDrive/tts_checkpoints/

# LoRA weights
print("\nLoRA adapters:")
!ls -lh /content/drive/MyDrive/tts_checkpoints/lora/

## 15. Inference Test (Ses Üretimi)

Training bittikten sonra modeli test edin

In [ ]:
# Test cümleleri
test_texts = [
    "Merhaba, size nasıl yardımcı olabilirim?",
    "Bugün hava çok güzel.",
    "Türkçe metin seslendirilmesi başarıyla çalışıyor.",
    "Bu bir test cümlesidir.",
]

# Her cümle için ses üret
for i, text in enumerate(test_texts):
    print(f"\nGenerating audio {i+1}/{len(test_texts)}: {text}")
    !python -m src.inference \
        --checkpoint /content/drive/MyDrive/tts_checkpoints/best_model.pt \
        --text "{text}"

# Üretilen dosyaları Drive'a kopyala
!cp outputs/*.wav /content/drive/MyDrive/tts_outputs/

print("\n✓ Audio files saved to Drive: /content/drive/MyDrive/tts_outputs/")

## 16. Ses Dosyalarını Dinle (Colab'da)

In [ ]:
# Üretilen sesleri Colab'da dinle
import IPython.display as ipd
import glob

audio_files = sorted(glob.glob('/content/drive/MyDrive/tts_outputs/*.wav'))

for audio_file in audio_files[-5:]:  # Son 5 dosya
    print(f"\n{audio_file}")
    display(ipd.Audio(audio_file))

## 17. Training İstatistikleri

In [ ]:
# TensorBoard loglarından training istatistiklerini çek
from tensorboard.backend.event_processing import event_accumulator
import glob

# En son log dosyasını bul
log_files = glob.glob('/content/drive/MyDrive/tts_runs/events.out.tfevents.*')

if log_files:
    latest_log = max(log_files, key=lambda x: os.path.getmtime(x))
    print(f"Reading log: {latest_log}\n")
    
    ea = event_accumulator.EventAccumulator(latest_log)
    ea.Reload()
    
    # Training loss
    if 'train/loss' in ea.Tags()['scalars']:
        train_loss = ea.Scalars('train/loss')
        print(f"Total training steps: {len(train_loss)}")
        print(f"Initial loss: {train_loss[0].value:.4f}")
        print(f"Final loss: {train_loss[-1].value:.4f}")
    
    # Validation loss
    if 'val/loss' in ea.Tags()['scalars']:
        val_loss = ea.Scalars('val/loss')
        print(f"\nBest validation loss: {min([x.value for x in val_loss]):.4f}")
else:
    print("No TensorBoard logs found")

## 18. Model'i Local'e İndir

Training bittikten sonra best model'i bilgisayarınıza indirin

In [ ]:
# Best model ve LoRA weights'i zip'le
import shutil

output_zip = '/content/drive/MyDrive/turkish_tts_model.zip'

# Geçici dizin oluştur
!mkdir -p /tmp/model_export

# Best model ve LoRA weights kopyala
!cp /content/drive/MyDrive/tts_checkpoints/best_model.pt /tmp/model_export/
!cp -r /content/drive/MyDrive/tts_checkpoints/lora/best_model /tmp/model_export/lora_weights
!cp config.yaml /tmp/model_export/
!cp data/phoneme_vocab.json /tmp/model_export/

# Zip oluştur
shutil.make_archive('/tmp/turkish_tts_model', 'zip', '/tmp/model_export')
!mv /tmp/turkish_tts_model.zip {output_zip}

print(f"\n✓ Model exported to: {output_zip}")
print(f"Download from Google Drive: MyDrive/turkish_tts_model.zip")
print(f"\nContents:")
!unzip -l {output_zip}

---

## Notlar ve İpuçları

### Training Süreci
- **Debug test**: Her zaman önce 100 sample ile test edin (hücre 9)
- **Checkpoint'ler**: Her 500 step'te Drive'a kaydedilir
- **TensorBoard**: Training sırasında loss'u izleyin (hücre 10)
- **Disconnect**: Olursa hücre 13 ile devam edin

### GPU Kullanımı
- **T4 GPU**: Batch size 8 optimal
- **V100/A100**: Batch size 16-32'ye çıkabilir
- **Out of Memory**: Batch size'ı azaltın (config.yaml)

### Sonraki Adımlar
1. Training bitince model'i indirin (hücre 18)
2. Local'de inference test yapın
3. İsterseniz başka veri seti ile devam edin

### Sorun Giderme
- **espeak-ng hatası**: Hücre 4'ü tekrar çalıştırın
- **Dataset inmiyor**: Cache temizleyin: `!rm -rf ~/.cache/huggingface`
- **CUDA out of memory**: Batch size azaltın veya High-RAM runtime kullanın
- **Disconnect sık oluyor**: Colab Pro deneyin (aylık ~10$)
